# Part1: Collecting Data from Skeleton Info

Import related packages

In [1]:
import pandas as pd
import numpy as np
import os
import json
import csv
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Bidirectional, Embedding, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as kb
from keras import initializers
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
#!pip install python_highcharts
from highcharts import Highchart
import json

Using TensorFlow backend.


Function Introduction: 

    1: getJsonList: given a folder path, read file names from this folder and return a list
    
    2: getJsonForOneFrame: given a json file name, which is the information from a frame, return one person's information with 190 data points as a list describing face and body
    
    3: Processing one json file for a frame, if there are multiple people, we use this function to select the person with the most data points 

# Part5 Testing on new video and generate outcomes

In [2]:
def getJsonList(filePath):
    fileList = []
    for i,j,k in os.walk(filePath):
        fileList = k
    return fileList
def calcWeight(item):
    zeros = 0
    for i in item['pose_keypoints_2d']:
        if i == 0:
            zeros += 1
    for i in item['face_keypoints_2d']:
        if i == 0:
            zeros +=1
    return zeros

def getJsonForOneFrame(fileName, filePath):
    l = getJsonList(filePath)
    with open(fileName,'r') as f:
        temp = json.loads(f.read())
        allPeople = temp['people']
    if len(allPeople) == 0:
        empty = []
        for i in range(190):
            empty.append(0)
        return empty
    target = allPeople[0]
    for dic in allPeople:
        if calcWeight(dic) < calcWeight(target):
            target = dic
    matrix = []
    if(len(target['pose_keypoints_2d']) == 0):
        for i in range(50):
            matrix.append(0)
    else:
        for i in range(75):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['pose_keypoints_2d'][i])
    if(len(target['face_keypoints_2d']) == 0):
        for i in range(140):
            matrix.append(0)
    else:
        for i in range(210):
            if (i+1) % 3 == 0:
                pass
            else:
                matrix.append(target['face_keypoints_2d'][i])
    return matrix


def writeData(path, pathType,fullDfList):
    fileList = getJsonList(path)
    fileList = set(fileList)
    print(len(fileList))
    dataCat = pd.read_csv('data.csv')
    for index, row in dataCat.iterrows():
        temp = int(row['name'][:-1])
        if(row["type"] != pathType):
            pass
        else:
            startName = row["name"]+"_"+'0'.zfill(12)+"_keypoints.json"
            endName = row["name"]+"_"+'30'.zfill(12)+"_keypoints.json"
            if endName and endName not in fileList:
                print('Video ',row["name"]," is not long enough")
            else:
                initFrame = 0
                while(row["name"]+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
                    matrix = []
                    for i in range(30):
                        if (i+1)% 5 == 0:
                            file = row["name"]+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
                            matrixLine = getJsonForOneFrame(path+'/'+file,path)
                            matrix.append(matrixLine)
                        else:
                            pass
                    if row['type'] == 'shortyes':
                        label = True
                    elif row['type'] == 'shortno':
                        label = False
                    else:
                        label = getLable(dataCat,row['name'], initFrame)
                    data = [[label, matrix]]
                    print("Data:",label, row['name'], initFrame,'matrix:',len(matrix),len(matrix[0]))
                    df = pd.DataFrame(data,columns=['lable','matrix'])
                    fullDfList.append(df)
                    initFrame += 30
    return fullDfList

def draw(folderName,videoName, modelName):
    #videoName = '6'#define your test video name here
    fileList = getJsonList(folderName)
    fileList = set(fileList)
    print(len(fileList))
    fullData = []
    initFrame = 0
    while(folderName+"_"+str(initFrame + 29).zfill(12)+"_keypoints.json" in fileList):
        matrix = []
        for i in range(30):
            file = folderName+"_"+str(initFrame + i).zfill(12)+"_keypoints.json"
            matrixLine = getJsonForOneFrame(folderName+'/'+file,folderName)

            matrix.append(matrixLine)
        fullData.append(matrix)
        initFrame += 30
    def cleanX(raw):
        length = len(raw)
        temp = np.zeros((length, 10,190))
        for i in range(len(raw)):
            for j in range(10):
                for k in range(190):
                    temp[i][j][k] = raw[i][j][k]
            print("progress:{0}%".format(round((i + 1) * 100 / len(raw))), end="\r")
        return temp
    temp = cleanX(fullData)
    model = load_model(modelName)
    predict = model.predict(temp)
    a = predict.tolist()
    result = []
    frames = 30
    for i in a:
        result.append([frames,i[0]])
        frames += 30
    #print(result)
    chart = Highchart()
    chart.set_options('chart', {'inverted': False})
    options = {
        'title': {
            'text': 'Prediction for video '+folderName+'.mp4'
        },
        'subtitle': {
            'text': '1 means talking while 0 means non-talking'
        },
         'xAxis': {
            'title': {
                'text': 'Second'
            }
        },
        'yAxis': {
            'title': {
                'text': 'Flag'
            },
        }
    }
    chart.set_dict_options(options)
    chart.add_data_set(result, series_type='line', name='prediction')
    
    
    
    index = []
    for i in result:
        for j in range(30):
            index.append(i[1])
            
        
    import cv2
    video = videoName
    result_video = folderName+'_result.mp4'
    cap = cv2.VideoCapture(video)
    fps_video = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    videoWriter = cv2.VideoWriter(result_video, fourcc, fps_video, (frame_width, frame_height))
    frame_id = 0
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            frame_id += 1
            left_x_up = int(frame_width / frame_id)
            left_y_up = int(frame_height / frame_id)
            right_x_down = int(left_x_up + frame_width / 10)
            right_y_down = int(left_y_up + frame_height / 10)
            word_x = left_x_up + 5
            word_y = left_y_up + 25
            if frame_id >= len(index):
                pass
            else:
                cv2.putText(frame, 'frame_%s' %index[frame_id], (word_x, word_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (55,255,155), 2)
            videoWriter.write(frame)
        else:
            videoWriter.release()
            break    
    
    frameRate = 30#default video frame rate
    for i in result:
        i[0] = i[0]/frameRate
    print(result)
    
    d = {}
    d['talking'] = result
    file_name = 'timeLabel_'+videoName+'.json' 
    with open(file_name,'w') as file_object:
        json.dump(d,file_object)
    return chart

In [3]:
folder_name = '5'#type your folder path here, which contains openpose json files
video_name = '5.mp4'#type your video path here, we want to know frame rate to create time labels 
model_name = 'balancetalking.h5'#type your model name here
chart1 = draw(folder_name,video_name,model_name)#return a frame/label chart and saved a time/label json file
chart1.save_file('chart_for_video_'+video_name)#save the chart 
chart1

1816
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
[[1.0, 0.030747830867767334], [2.0, 0.03154382109642029], [3.0, 0.0309392511844635], [4.0, 0.03367513418197632], [5.0, 0.031129121780395508], [6.0, 0.029649466276168823], [7.0, 0.029647350311279297], [8.0, 0.02960565686225891], [9.0, 0.0303460955619812], [10.0, 0.0297127366065979], [11.0, 0.031129777431488037], [12.0, 0.03185218572616577], [13.0, 0.029551416635513306], [14.0, 0.029703468084335327], [15.0, 0.03013703227043152], [16.0, 0.03323683142662048], [17.0, 0.02925664186477661], [18.0, 0.0555652379989624], [19.0, 0.03450074791908264], [20.0, 0.029789119958877563], [21.0, 0.030101925134658813], [22.0, 0.029540449380874634], [23.0, 0.030952751636505127], [24.0, 0.03114628791809082], [25.0, 0.030035585165023804], [26.0, 0.028204232454299927], [27.0, 0.02820301055908203], [28.0, 0.02765229344367981], [29.0, 0.02684265375137329], [30.0, 0.029056280851364136], [31